In [2]:
# !pip install -U langchain-cli
# !pip install -U "langchain[all]"
# !pip install pypdf
# !pip install chroma

Create the app

In [3]:
from dotenv import load_dotenv

load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
# os.environ["OPENAI_API_KEY"] = "your-api-key"

In [4]:
!yes|langchain app new chat-with-pdf --package rag-chroma

ModuleNotFoundError: No module named 'pexpect'

In [ ]:
!ls ./chat-with-pdf

Dockerfile     README.md      app            packages       pyproject.toml


In [ ]:
!ls ./chat-with-pdf/app


__init__.py __pycache__ server.py


Find the `/app/server.py` file and add:

```
from rag_conversation import chain as rag_chroma_chain

add_routes(app, rag_chroma_chain, path="/rag-chroma")
```

Optionally you can configure langsmith:

```
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=<your-api-key>
export LANGCHAIN_PROJECT=<your-project>  # if not specified, defaults to "default"
```

Now, let's add some data to chat with!

For that let's open the chain.py file that was created in `/packages/rag-chroma/rag-chroma/chain.py`

Let's modify it to accept data releavnt to us, in our case let's set up a chat with a blog article about agents:

- https://lilianweng.github.io/posts/2023-06-23-agent/

```
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Add to vectorDB
vectorstore = Chroma.from_documents(documents=all_splits, 
                                    collection_name="rag-chroma",
                                    embedding=OpenAIEmbeddings(),
                                    )
retriever = vectorstore.as_retriever()
```

The rest of the code should remain the same.

Now, we can deploy our app:

In [ ]:
! cd chat-with-pdf && langchain serve

INFO:     Will watch for changes in these directories: ['/Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_agents/notebooks/chat-with-pdf']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [4574] using WatchFiles
/Users/greatmaster/miniconda3/envs/oreilly-langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/greatmaster/miniconda3/envs/oreilly-langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_com

Let's access our chat by going to:

http://localhost:8000/rag-chroma/playground

To access the the template from code run this:
(make sure to serve the app in a terminal first with the command shown previously).

In [ ]:
from langserve.client import RemoteRunnable

runnable = RemoteRunnable("http://localhost:8000/rag-chroma")

In [ ]:
runnable

In [ ]:
runnable.invoke("Write a summary of these papers")

'The given context provides information about several papers related to the topic of summarization. However, it does not directly contain a summary of these papers.'